In [41]:
from importlib import reload 
import datakit
reload(datakit)
from datakit import *

pd.set_option('display.max_columns', None)

In [43]:
df = pd.read_csv('data_raw/otodom_last7_2024_04_18.csv')

In [3]:
subways_location = {
    'Plac Wilsona': {'Latitude': 52.26902, 'Longitude': 20.985},
    'Wilanowska': {'Latitude': 52.18098, 'Longitude': 21.02364},
    'Centrum': {'Latitude': 52.23055, 'Longitude': 21.01091},
    'Stokłosy': {'Latitude': 52.15583, 'Longitude': 21.0347},
    'Wierzbno': {'Latitude': 52.18932, 'Longitude': 21.01726},
    'Pole Mokotowskie': {'Latitude': 52.2079, 'Longitude': 21.00837},
    'Swiętokrzyska': {'Latitude': 52.23499, 'Longitude': 21.00835},
    'Ratusz': {'Latitude': 52.24476, 'Longitude': 21.00082},
    'Marymont': {'Latitude': 52.27178, 'Longitude': 20.97232},
    'Imielin': {'Latitude': 52.14952, 'Longitude': 21.04543},
    'Kabaty': {'Latitude': 52.13109, 'Longitude': 21.06604},
    'Ursynów': {'Latitude': 52.16217, 'Longitude': 21.02858},
    'Natolin': {'Latitude': 52.14033, 'Longitude': 21.05776},
    'Bemowo': {'Latitude': 52.2393879, 'Longitude': 20.9175868},
    'Ulrychów': {'Latitude': 52.2402211, 'Longitude': 20.9286984},
    'Księcia Janusza': {'Latitude': 52.238989, 'Longitude': 20.9413589},
    'Młynów': {'Latitude': 52.2370368, 'Longitude': 20.9582514},
    'Płocka': {'Latitude': 52.2331322, 'Longitude': 20.9672229},
    'Rondo Daszyńskiego': {'Latitude': 52.2302503, 'Longitude': 20.9812049},
    'Rondo ONZ': {'Latitude': 52.2326673, 'Longitude': 20.9943419},
    'Nowy Świat Uniwersytet': {'Latitude': 52.2372522, 'Longitude': 21.0163951},
    'Centrum Nauki Kopernik': {'Latitude': 52.2405898, 'Longitude': 21.029495},
    'Stadion Narodowy': {'Latitude': 52.2468026, 'Longitude': 21.0437104},
    'Wileński': {'Latitude': 52.2552165, 'Longitude': 21.0373375},
    'Szwedzka': {'Latitude': 52.2658294, 'Longitude': 21.0477777},
    'Targówek+Mieszkaniowy': {'Latitude': 52.278611, 'Longitude': 21.0478345},
    'Zacisze': {'Latitude': 52.278611, 'Longitude': 21.0478345},
    'Kondratowicza': {'Latitude': 52.2844033, 'Longitude': 21.0489269},
    'Bródno': {'Latitude': 52.2844033, 'Longitude': 21.0489269}}

In [45]:
df_1 = scrub_data(df, subways_location)

In [8]:
df_1[['rent_price', 'year_of_construction']].copy().dropna().to_excel('test.xlsx')

In [47]:
df_1.head(5)

,rent_price,additional_fees,area,room_num,floor,students_allowed,elevator,parking_space,year_of_construction,additional_information,latitude,longitude,approximate_coordinates,adv_description,link,district,subway_distance,center_distance,bt_apartment,bt_tenement,bt_block,bt_other,is_furnished,dishwasher,air_conditioning,for_renovation,at_private,at_agency,at_developer,balcony,terrace,garden,cy_old_building,cy_new_building,cy_other,building_height
0,2500.0,650.0,35.0,1,4.0,False,True,True,1990.0,oddzielna kuchnia,52.184479,21.046109,False,"Witam,\nna wynajem posiadam kawalerkę o powier...",https://www.otodom.pl/pl/oferta/kawalerka-35m2...,Mokotow,1.581,5.534,False,False,True,False,True,False,False,False,True,False,False,True,True,False,False,False,True,10.0
1,3200.0,550.0,40.0,3,1.0,True,False,False,1980.0,NaN,52.296203,21.039305,False,Bródno ul. Łojewska 7\n3-pokojowe mieszkanie p...,https://www.otodom.pl/pl/oferta/mieszkanie-3-o...,Targowek,1.466,7.621,False,False,True,False,True,True,False,False,True,False,False,False,False,False,False,False,True,10.0
2,3000.0,700.0,46.0,2,10.0,True,True,False,1970.0,"piwnica, tylko dla niepalących, oddzielna kuch...",52.238405,21.100078,False,Mam do wynajęcia mieszkanie 2 pokojowe z oddzi...,https://www.otodom.pl/pl/oferta/wynajme-mieszk...,Praga_Poludnie,3.950,6.062,False,False,True,False,True,True,False,False,True,False,False,True,False,False,False,False,True,15.0
3,3500.0,600.0,47.0,3,2.0,False,False,False,1965.0,"piwnica, oddzielna kuchnia",52.199612,21.031665,False,Do wynajęcia 3 pokojowe mieszkanie na Dolnym M...,https://www.otodom.pl/pl/oferta/3-pokoje-z-bal...,Mokotow,1.508,3.599,False,False,True,False,True,False,False,False,True,False,False,True,False,False,False,False,True,4.0
4,2500.0,600.0,36.0,2,1.0,True,False,False,1958.0,"tylko dla niepalących, oddzielna kuchnia",52.280803,20.930511,True,Do wynajęcia 2-pokojowe mieszkanie w odremonto...,https://www.otodom.pl/pl/oferta/stare-bielany-...,Bielany,3.016,7.950,False,True,False,False,True,False,False,False,False,True,False,True,False,False,False,False,True,4.0


In [51]:
df_1.dtypes

rent_price                 float64
additional_fees            float64
area                       float64
room_num                    object
floor                      float64
students_allowed              bool
elevator                      bool
parking_space                 bool
year_of_construction       float64
additional_information      object
latitude                   float64
longitude                  float64
approximate_coordinates       bool
adv_description             object
link                        object
district                    object
subway_distance            float64
center_distance            float64
bt_apartment                object
bt_tenement                 object
bt_block                    object
bt_other                      bool
is_furnished                  bool
dishwasher                    bool
air_conditioning              bool
for_renovation                bool
at_private                    bool
at_agency                     bool
at_developer        

In [15]:
df.floor.unique()

array(['4/10', '1/10', '> 10/15', '2/4', 'parter/4', '4/5', '1', '9/11',
       '7/8', '3', '1/3', '9/10', '2/5', nan, '3/11', '3/5', '2/7', '2/3',
       '3/4', '7/11', '5/10', 'parter/3', 'parter/5', '4/4', '3/3',
       '6/14', 'parter/15', '6/7', 'parter/7', '4/7', '5/6', '5/11',
       '2/2', '7/9', '10/17', '> 10/52', '5/5', '2/6', '8/12', '1/7', '4',
       '6/6', '1/6', '1/2', '6/8', '5/8', '7/7', '1/5', '7/20', '3/6',
       '1/4', '3/8', '2/10', '5', 'parter/10', '9/9', '> 10/11', '5/7',
       'parter', '3/7', '> 10/13', '1/8', '10/20', '4/9', '5/17', '2',
       '2/9', '4/6', '8/8', '6/17', '3/9', '10/11', '> 10/25', '4/12',
       '7/14', '8/10', '4/8', '3/2', 'parter/6', '10/12', '8/9', '2/8',
       '2/11', '9/16', 'parter/2', '10', '7/10', '7', 'parter/11', '7/25',
       '8/15', '9/13', '10/10', '6/9', '5/14', '4/14', '10/5', '1/9',
       '> 10/29', '6/10', '> 10/28', '> 10/18', '6/11', '1/15',
       'parter/1', 'parter/8', '> 10', '5/9', '8/11', '3/10', '10/16',
   

In [25]:
df[df.floor.isna()]

,rent_price,additional_fees,location,area,room_num,floor,building_type,extra_space,flat_condition,advertiser_type,students_allowed,furnishings,elevator,parking_space,year_of_construction,additional_information,latitude,longitude,approximate_coordinates,adv_description,link,cy_old_building,cy_new_building,cy_other
17,2 600 zł,750 zł/miesiąc,"ul. Aleksandra Świętochowskiego, Jelonki Półno...",48 m²,2,NaN,NaN,NaN,NaN,prywatny,NaN,NaN,nie,NaN,NaN,NaN,52.234909,20.897288,False,Na wynajem 2-pokojowe mieszkanie o powierzchni...,https://www.otodom.pl/pl/oferta/2-pokoje-46-m2...,False,False,True
19,2 800 zł,600 zł/miesiąc,"ul. Adolfa Pawińskiego, Rakowiec, Ochota, Wars...",36 m²,2,NaN,blok,NaN,NaN,prywatny,NaN,"zmywarka, lodówka, meble, piekarnik, kuchenka,...",nie,NaN,NaN,NaN,52.205799,20.977109,False,na wynajem 2-pokojowe mieszkanie o powierzchni...,https://www.otodom.pl/pl/oferta/ochota-niezale...,False,False,True
110,2 800 zł,NaN,"ul. Wolska 171, Odolany, Wola, Warszawa, mazow...",22 m²,1,NaN,apartamentowiec,NaN,do zamieszkania,prywatny,NaN,"zmywarka, lodówka, meble, kuchenka, pralka",tak,NaN,2022.0,"klimatyzacja, tylko dla niepalących",52.225534,20.939992,False,[English version below]\nDo wynajęcia kawalerk...,https://www.otodom.pl/pl/oferta/kawalerka-na-w...,False,True,False
140,4 000 zł,1 300 zł/miesiąc,"ul. Stokłosy, Ursynów Północny, Ursynów, Warsz...",87 m²,4,NaN,NaN,NaN,NaN,biuro nieruchomości,NaN,NaN,nie,NaN,NaN,NaN,52.156076,21.034723,False,Mieszkanie dla małżeństwa z dziećmi lub studen...,https://www.otodom.pl/pl/oferta/4-pokoje-87m2-...,False,False,True
147,3 200 zł,800 zł/miesiąc,"ul. Instalatorów 5a, Raków, Włochy, Warszawa, ...",45 m²,2,NaN,NaN,balkon,NaN,prywatny,NaN,"zmywarka, lodówka, meble, piekarnik, kuchenka,...",tak,NaN,NaN,tylko dla niepalących,52.199080,20.954572,False,"Wynajmę mieszkanie 2 pokojowe 45 m2, wysoki st...",https://www.otodom.pl/pl/oferta/mieszkanie-2-p...,False,False,True
152,3 000 zł,500 zł/miesiąc,"ul. Bieniewicka, Marymont-Potok, Żoliborz, War...","36,24 m²",1,NaN,blok,NaN,NaN,biuro nieruchomości,NaN,meble,nie,NaN,1954.0,oddzielna kuchnia,52.273138,20.972744,False,Mieszkanie 1-pokojowe na parterze w 4 piętrowe...,https://www.otodom.pl/pl/oferta/jednopokojowe-...,False,False,True
220,4 900 zł,NaN,"Zacisze, Targówek, Warszawa, mazowieckie",90 m²,3,NaN,NaN,NaN,NaN,biuro nieruchomości,NaN,NaN,nie,NaN,NaN,NaN,52.285222,21.032300,True,"Do wynajęcia mieszkanie 3pok, 90m2, z balkonem...",https://www.otodom.pl/pl/oferta/eleganckie-3po...,False,False,True
304,2 800 zł,600 zł/miesiąc,"Chrzanów, Bemowo, Warszawa, mazowieckie",37 m²,2,NaN,NaN,NaN,NaN,prywatny,NaN,NaN,nie,NaN,NaN,NaN,52.224378,20.864856,False,"Mieszkanie składa się z dwóch odrębnych pokoi,...",https://www.otodom.pl/pl/oferta/warszawa-bemow...,False,False,True
318,3 200 zł,570 zł/miesiąc,"ul. Antoniego Malczewskiego, Wierzbno, Mokotów...",52 m²,2,NaN,NaN,NaN,NaN,biuro nieruchomości,NaN,NaN,nie,NaN,NaN,NaN,52.193440,21.016060,False,We speak PL | EN | RUS\n\n \nPolski:\nZaprasza...,https://www.otodom.pl/pl/oferta/mieszkanie-2-p...,False,False,True
363,3 300 zł,500 zł/miesiąc,"ul. Żupnicza, Kamionek, Praga-Południe, Warsza...",44 m²,2,NaN,apartamentowiec,balkon,do zamieszkania,biuro nieruchomości,NaN,"zmywarka, lodówka, piekarnik, kuchenka, pralka",tak,garaż/miejsce parkingowe,2023.0,NaN,52.251943,21.062320,False,Mieszkanie bez mebli w pokojach.\nNowe dopiero...,https://www.otodom.pl/pl/oferta/mieszkanie-44-...,False,True,False


In [11]:
df['cy_old_building'] = np.where(df['year_of_construction'] <= 1950, True, False)
df['cy_new_building'] = np.where(df['year_of_construction'] >= 2000, True, False)
df['cy_other'] = np.where(~df['cy_old_building'] & ~df['cy_new_building'], True, False)

In [11]:
df_1.year_of_construction.min()

11.0

In [38]:
~df_1.parking_space.isna()

0        True
1       False
2       False
3       False
4       False
        ...  
1187     True
1188    False
1189     True
1190     True
1191     True
Name: parking_space, Length: 1191, dtype: bool

In [30]:
type('balkon')

str

In [34]:
df['balcony'] = df['extra_space'].apply(lambda x: True if type(x)==str and 'balkon' in x else False)
df['terrace'] = df['extra_space'].apply(lambda x: True if type(x)==str and 'taras' in x else False)
df['garden'] = df['extra_space'].apply(lambda x: True if type(x)==str and 'ogródek' in x else False)

0        True
1       False
2        True
3        True
4        True
        ...  
1205    False
1206     True
1207     True
1208     True
1209     True
Name: extra_space, Length: 1210, dtype: bool

In [35]:
df.advertiser_type.str.contains('biuro nieruchomości')

0       False
1       False
2       False
3       False
4        True
        ...  
1205     True
1206    False
1207    False
1208     True
1209     True
Name: advertiser_type, Length: 1210, dtype: bool